In [60]:
import faiss
import os
import json
import numpy as np

files = os.listdir('/home/wagyu0923/project/Document_Analyzer/embedded_files')

meta_datas = []
embeddings = []

for file in files:
    with open(f'/home/wagyu0923/project/Document_Analyzer/embedded_files/{file}', 'r') as f:
        embedded_text = json.load(f)
    
    
    for data in embedded_text:
        meta_data = {}
        meta_data['source'] = file
        meta_data['page'] = data['page']
        meta_data['chunk'] = data['chunk']
        meta_datas.append(meta_data)
        embeddings.append(data['embedding'])

embeddings = np.array(embeddings, dtype = 'float32')



In [61]:
dimension = len(embeddings[0])
index = faiss.IndexFlatL2(dimension)

index.add(embeddings)



In [62]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('intfloat/multilingual-e5-large-instruct')


In [70]:
query = input()
query_vector = model.encode(query)

distances, indices = index.search(np.array([query_vector],dtype = 'float32'), k=5)

retrived_data = ''
for idx in indices[0]:
    temp = str(meta_datas[idx])
    retrived_data += temp


In [74]:
import ollama

OLLAMA_MODEL = 'hf.co/unsloth/gpt-oss-20b-GGUF:Q4_K_M'
SYSTEM_PROMPT = f"""You are an intelligent assistant helping the users with their questions on company. Strictly Use ONLY the following pieces of context to answer the question at the end. 

Do not try to make up an answer:
 - 우선 인식된 정보를 정리해서 말할 것
 - If the answer to the question cannot be determined from the context alone, say "I cannot determine the answer to that." and the reason why you cannot determine
 - If the context is empty, just say "I do not know the answer to that." 
 - CONTEXT에 주어진 내용을 토대로 추론해서 QUESTION에 근거를 대며 상세히 답할 것.
 - Answer in Korean
CONTEXT: 
{retrived_data}
QUESTION:
{query}
"""

response = ollama.chat(
    model=OLLAMA_MODEL,
    messages=[
        {'role': 'user', 'content': SYSTEM_PROMPT}
    ],
    options={
        'num_predict': 8192,
        'temperature': 0.1,
        'repeat_penalty': 1.1,
    }
)
outputs = response['message']['content']

In [75]:
print(outputs)

세토피아가 직면하고 있는 투자 리스크는 다음과 같이 세 가지로 요약될 수 있습니다.  

| # | 리스크 종류 | 근거(문맥) | 구체적 내용 |
|---|--------------|------------|------------|
| 1 | **유동성(유동성) 리스크** | *page 117* – “부채 유동성 대체 73,070 113,738” | 부채의 유동성 비율이 낮아 단기 채무를 이행하기 어려울 가능성. |
| 2 | **규제·법적 리스크** | *page 214* – “과징금 1억7,680만원, 과태료 3억… 과징금 27억원” 등 다수의 벌금·과태료가 부과된 기록 | 자본시장·금융투자업법, 외국환거래법 등 관련 법령 위반 사례가 다수 존재해 향후 추가 제재 가능성. |
| 3 | **회계·재무제표 리스크** | *page 214* – “회계처리위반이 확정될 경우 회계장부 및 재무제표를 반영하여 재공시 예정” | 회계처리 부정확인 시 재무제표에 반영될 필요가 있어 재무제표 신뢰성에 대한 리스크. |

이 세 가지 리스크는 투자자에게 재무적 안정성, 규제 준수 여부, 그리고 단기 유동성 확보 측면에서 주의적 위험 요소로 작용할 수 있습니다.


In [73]:
print(retrived_data)

{'source': '[세토피아][정정]반기보고서(2025.09.09)_chunk_embedding.json', 'page': 117, 'chunk': '부채 유동성 대체 73,070 113,738'}{'source': '[세토피아][정정]반기보고서(2025.09.09)_chunk_embedding.json', 'page': 70, 'chunk': '942) 비지배지분 - - - 기본주당이익(손실) (354) (358) (442) 희석주당이익(손실) (354) (358) (443) 연결에 포함된 회사수 4 6 6 주식회사 세토피아 (단위 : 원) 과 목 제11(당반)기말 제10(전)기말 제9(전전)기말 [유동자산] 8,536,561,939 10,133,404,461 37,541,162,024 ㆍ현금및현금성자산 457,702,865 182,561,012 4,936,849,113 ㆍ매출채권 1,881,162,022 3,892,987,351 14,037,824,167 ㆍ계약자산 - - 169,161,731 ㆍ단기기타채권 4,462,330,881 2,067,748,000 8,574,471,117 ㆍ기타유동자산 155,254,977 896,980,649 1,203,248,673 ㆍ파생상품자산 57,923,770 190,681,348 520,935,485 ㆍ당기법인세자산 94,950 1,654,500 26,061,520 ㆍ재고자산 1,522,092,474 2,900,791,601 7,726,861,555 ㆍ매각예정자산 - - 345,748,663 [비유동자산] 16,500,806,760 21,121,414,827 31,315,005,805 ㆍ당기손익-공정가치측정금융자산 13,733,387 47,297,225 388,483,452 ㆍ장기기타채권 471,589,000 483,463,000 3,'}{'source': '[세토피아][정정]반기보고서(2025.09.09)_chunk_embedding.json', 'page': 155, 'chunk': '초 발행 상환 등 평가 기타